In [12]:
import gzip
import pickle
from tensorflow import keras
import numpy as np
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, BatchNormalization, \
                                    Activation, Dropout, MaxPooling2D

def evaluate(path, model):
    X, y = pickle.load(gzip.open(path, 'rb'))
    y[y != 0] -= 2
    X = X / 255.
    acc = np.mean(model(X).numpy().argmax(axis=1) == y)
    return acc


In [13]:
path = 'flatland_train.data'
X,y = pickle.load(gzip.open(path,'rb')) 

# data pre-proc
y[y != 0] -= 2
X = X.reshape (X.shape[0],50,50,1) / 255.

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 42)

DNN modelis

In [15]:
# model = keras.models.Sequential()
# model.add(keras.layers.Flatten(input_shape=[50, 50]))
# model.add(keras.layers.Dense(64, activation="relu"))
# model.add(keras.layers.Dense(32, activation="relu"))
# model.add(keras.layers.Dense(10, activation="softmax"))
# model.compile(loss="sparse_categorical_crossentropy",
#               optimizer="Adam",
#               metrics=["accuracy"])
# model.summary()

CNN modelis

In [16]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[50, 50, 1]))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_2 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 22, 22, 32)        9248      
                                                                 
 batch_normalization_3 (Batc  (None, 22, 22, 32)       128       
 hNormalization)                                                 
                                                      

In [17]:
loss = model.fit(X_train, y_train, epochs=20,validation_data=[X_test,y_test])


Epoch 1/20
219/219 [==============================] - 4s 13ms/step - loss: 1.1648 - accuracy: 0.5601 - val_loss: 10.6879 - val_accuracy: 0.1520
Epoch 2/20
219/219 [==============================] - 3s 12ms/step - loss: 0.4708 - accuracy: 0.8349 - val_loss: 9.8707 - val_accuracy: 0.1520
Epoch 3/20
219/219 [==============================] - 3s 12ms/step - loss: 0.2485 - accuracy: 0.9306 - val_loss: 0.8566 - val_accuracy: 0.6300
Epoch 4/20
219/219 [==============================] - 3s 12ms/step - loss: 0.1530 - accuracy: 0.9584 - val_loss: 0.4294 - val_accuracy: 0.8500
Epoch 5/20
219/219 [==============================] - 3s 12ms/step - loss: 0.1066 - accuracy: 0.9723 - val_loss: 0.3632 - val_accuracy: 0.9053
Epoch 6/20
219/219 [==============================] - 2s 11ms/step - loss: 0.0848 - accuracy: 0.9757 - val_loss: 0.2908 - val_accuracy: 0.9360
Epoch 7/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0691 - accuracy: 0.9790 - val_loss: 0.3226 - val_accuracy: 0.929

In [18]:
model.save('model_cnn6.h5')


In [10]:
model = keras.models.load_model('model_cnn6.h5')
evaluate('flatland_train.data', model)

0.9831

In [11]:
pred = model.predict(X_test).argmax(axis=1)
print('Accuracy on test set - {0:.02%}'.format((pred == y_test).mean()))

Accuracy on test set - 95.40%
